In [1]:
import pandas as pd
import numpy as np

In [2]:
f = open('10288320_anomaly_rm.txt')
data = f.readlines()

In [3]:
stopword = 'a, are, an, the, and, or, but, if, then, because, about, above, after, all, also, although, am, an, and, any, as, at, be, because, been, before, being, between, both, but, by, came, can, come, could, did, do, does, each, else, for, from, get, got, had, has, have, he, her, here, him, himself, his, how, i, if, in, into, is, it, its, just, like, make, many, me, might, more, most, much, must, my, never, no, nor, not, now, of, on, only, or, other, our, out, over, said, same, see, should, since, so, some, still, such, take, than, that, the, their, them, then, there, these, they, this, those, through, to, too, under, up, use, very, want, was, way, we, well, were, what, when, where, which, while, who, will, with, would, you, your'.replace(' ','').split(',')

In [5]:
library_headline = dict()
library_short_description = dict()
#word_class_count_head = dict()#단어가 key 인 dict로 각 단어는 classes value들을 가지고 있다.
#word_class_count_desc = dict()#단어가 key 인 dict로 각 단어는 classes value들을 가지고 있다.
word_set_headline = set()
word_set_desc = set()
classes = []

for i in data[:int(len(data)*0.7)]:
    token = i.replace('\n','').lower().split('\t')
    if token[2] not in classes:
        classes.append(token[2])

for i in classes:
    library_headline[i] = dict()
    library_short_description[i] = dict()

for i in data[:int(len(data)*0.7)]:
    token = i.replace('\n','').lower().split('\t')
    head_split = token[1].split()
    desc_split = token[3].split()
    for k in head_split:
        if k in stopword:
            continue
        word_set_headline.add(k)
        if k not in library_headline[token[2]]:
            library_headline[token[2]][k] = 0
        else:
            library_headline[token[2]][k] +=1
            
    for k in desc_split:
        if k in stopword:
            continue
        word_set_desc.add(k)
        if k not in library_short_description[token[2]]:
            library_short_description[token[2]][k] = 0
        else:
            library_short_description[token[2]][k] +=1


In [6]:
del library_short_description['category']
del library_headline['category']

In [7]:
library_headline = dict(sorted(library_headline.items(), key = lambda item: len(item[1]), reverse = True))
library_short_description = dict(sorted(library_short_description.items(), key = lambda item: len(item[1]), reverse = True))

In [8]:
headline_list = list(library_headline.keys())
desc_list = list(library_short_description.keys())

for i in headline_list:
    library_headline[i] = dict(sorted(library_headline[i].items(), key = lambda item: (item[1]), reverse = True))

for i in desc_list:
    library_short_description[i] = dict(sorted(library_short_description[i].items(), key = lambda item: (item[1]), reverse = True))

In [42]:
def naive_bayes(input_string, target_classes=library_headline, vocab=word_set_headline):
    pre_processed_string = input_string.replace('\r','').replace('\t','').lower().split()
    candidate_probability = dict()
    total_vocab_size = len(vocab)
    for i, v in target_classes.items():#각 클래스별 순회
        specific_class_sum = sum(v.values())
        candidate_probability[i] = 1

        for k in pre_processed_string:#문장안에 단어별로
            if k not in v:#문장에 있는 단어중 클래스 안에 존재하지 않는 워딩 처리
                candidate_probability[i] = candidate_probability[i] * (1) / (specific_class_sum + total_vocab_size)
            else:
                candidate_probability[i] = candidate_probability[i] * (v[k] + 1) / (specific_class_sum + total_vocab_size)
                
    top1 = list(sorted(candidate_probability.items(), key = lambda item: (item[1]), reverse = True))
    return top1, top1[0][0]
        
            

In [60]:
#Evaluation with test set
total_count = 0
correct_count = 0
for i in data[int(len(data)*0.7):]:
    prob,prediction = naive_bayes(i.split('\t')[1])
    if(i.split('\t')[2].lower() == prediction):
        correct_count =correct_count +1
    total_count = total_count +1    

In [61]:
print(correct_count / total_count * 100)

42.27129828493168


In [63]:
#Evaluation with test set
total_count = 0
correct_count = 0
for i in data[int(len(data)*0.7):]:
    prob,prediction = naive_bayes(i.split('\t')[1], target_classes=library_short_description, vocab=word_set_desc)
    if(i.split('\t')[2].lower() == prediction):
        correct_count =correct_count +1
    total_count = total_count +1    

print(correct_count / total_count * 100)

34.55815980328073
